##### In this notebook, we will try to use convolution-recurrent nural networks to solve sentiment classification problem. 

##### Data preprocessing

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5424696771953592275
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5892014080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16110596128807350260
physical_device_desc: "device: 0, name: GeForce GTX 980 Ti, pci bus id: 0000:09:00.0, compute capability: 5.2"
]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import TensorBoard
from time import time

In [5]:
train = pd.read_csv("./data/train.tsv", sep = "\t")
test = pd.read_csv("./data/test.tsv", sep = "\t")

In [6]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [7]:
train_text = train.Phrase
test_text = test.Phrase
total_reviews = list(train_text) + list(test_text)

In [8]:
tk = Tokenizer(lower=True, filters='')

In [9]:
tk.fit_on_texts(total_reviews)

##### The max length of the reviews is 56

In [10]:
max_length = max([len(s.split()) for s in total_reviews])

In [11]:
train_tokenized = tk.texts_to_sequences(train_text)
test_tokenized = tk.texts_to_sequences(test_text)

In [12]:
from keras.preprocessing.sequence import pad_sequences

In [13]:
y_train = train.Sentiment

##### The vocabulary size is 19479

In [14]:
vocab_size = len(tk.word_index) + 1

##### Padding each review to the same length

In [15]:
X_train = pad_sequences(train_tokenized, maxlen=max_length, padding='post')
X_test = pad_sequences(test_tokenized, maxlen=max_length, padding='post')

In [16]:
y = np.zeros([len(y_train), 5])

In [17]:
for i in range(len(y_train)):
    y[i,y_train[i]-1] = 1

In [18]:
y

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

The Embedding layer requires the specification of the vocabulary size **(vocab_size)**, the size of the real-valued vector space **EMBEDDING_DIM = 100**, and the maximum length of input documents **max_length** .

##### Start building the model

In [19]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [20]:
EMBEDDING_DIM = 100

In [21]:
def BuildModel():
    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length, name = 'embedding'))
    model.add(Conv1D(100, 10, activation='relu', name = 'conv1'))
    model.add(MaxPooling1D(pool_size=2, name = 'conv1_max'))
    model.add(LSTM(64, name = 'lstm'))
    model.add(Dropout(0.15, name = 'dropout'))
    model.add(Dense(5, activation='softmax', name = 'dense'))
    model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
    return model

In [22]:
model = BuildModel()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [27]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

Instructions for updating:
Use the retry module or similar alternatives.


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 56, 100)           1947900   
_________________________________________________________________
conv1 (Conv1D)               (None, 47, 100)           100100    
_________________________________________________________________
conv1_max (MaxPooling1D)     (None, 23, 100)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 2,090,565
Trainable params: 2,090,565
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(X_train, y, epochs = 1, batch_size=500, verbose=1, callbacks=[tensorboard])

Epoch 1/1
156060/156060 [==============================] - 17s 107us/step - loss: 1.1551 - acc: 0.5370


In [24]:
from ann_visualizer.visualize import ann_viz

In [35]:
ann_viz(model, title="My first neural network")

ValueError: ANN Visualizer: Layer not supported for visualizing